In [1]:
import numpy as np
from collections import deque
import numpy as np
import time
import pandas as pd

In [112]:
def matrix_to_tuple(matrix):
    return (matrix[0][1], matrix[0][2],
            matrix[1][2])

def tuple_to_matrix(my_tuple):
    return np.array([[my_tuple[0], my_tuple[1], my_tuple[2]], 
                     [my_tuple[3], my_tuple[4], my_tuple[5]], 
                     [my_tuple[6], my_tuple[7], my_tuple[8]]])

def apply_mat(mat, index):
    if index==0:
        return mat @ A
    elif index==1:
        return mat @ B
    elif index==2:
        return mat @ C
    elif index==3:
        return mat @ D
    assert(1==2)

# B is the inverse of A
A = np.array([[1, 1, 0], [0, 1, 0], [0, 0, 1]])
B = np.array([[1, -1, 0], [0, 1, 0], [0, 0, 1]])

# C is the inverse of D
C = np.array([[1, 0, 0], [0, 1, 1], [0, 0, 1]])
D = np.array([[1, 0, 0], [0, 1, -1], [0, 0, 1]])

# together, A, B, C, and D generate the heisenberg group

I = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

In [113]:
data = pd.DataFrame({
        'val1': [],
        'val2': [],
        'val3': [],
        'last_matrix': [],
        "num_steps": []
    })

I_as_tuple = matrix_to_tuple(I)

cur_row_df = pd.DataFrame([[
    I_as_tuple[0], I_as_tuple[1], I_as_tuple[2], 
    0, 0
    ]], columns=['val1', 'val2', 'val3', 'last_matrix', 'num_steps'])
data = pd.concat([cur_row_df, data])

In [114]:
data

,val1,val2,val3,last_matrix,num_steps
0,0.0,0.0,0.0,0.0,0.0


In [115]:
# adapted from ChatGPT
# visited contains keys of the tuples we've already visited and values of either 0 when A was the matrix that brought us to this state
# of 1 if B was the matrix that brought us to this state

visited = set()

start_time = time.time()

def bfs(start, myDF):
    visited.add(matrix_to_tuple(start))
    visited.add(matrix_to_tuple(start @ A))
    visited.add(matrix_to_tuple(start @ B))
    visited.add(matrix_to_tuple(start @ C))
    visited.add(matrix_to_tuple(start @ D))
    
    queue = deque([[A, 1, 0], [B, 1, 1], [C, 1, 2], [D, 1, 3]])

    while queue:
        cur_item = queue.popleft()
        cur_mat = cur_item[0]
        cur_index = cur_item[1]
        cur_neighbor = cur_item[2]
        cur_tuple = matrix_to_tuple(cur_mat)

        cur_row_df = pd.DataFrame([[
            cur_tuple[0], cur_tuple[1], cur_tuple[2], 
            cur_index, cur_neighbor
            ]], columns=['val1', 'val2', 'val3', 'num_steps', 'last_matrix'])
        myDF = pd.concat([cur_row_df, myDF])


        if time.time() >= start_time + 600:
            # set a point to end this, currently 10 minutes
            print("Timed out")
            return myDF

        for neighbor in [0, 1, 2, 3]:
            new_mat = apply_mat(cur_mat, neighbor)
            new_tuple = matrix_to_tuple(new_mat)
            if not new_tuple in visited:
                visited.add(new_tuple)
                queue.append([new_mat, cur_index+1, neighbor])
        

data = bfs(I, data)

Timed out


In [116]:
data=data.reset_index(drop=True)
data

,val1,val2,val3,num_steps,last_matrix
0,-23.0,66.0,-1.0,28.0,3.0
1,-24.0,45.0,0.0,28.0,3.0
2,-25.0,22.0,1.0,28.0,3.0
3,-26.0,-3.0,2.0,28.0,1.0
4,-2.0,89.0,-8.0,28.0,0.0
...,...,...,...,...,...
255581,0.0,0.0,-1.0,1.0,3.0
255582,0.0,0.0,1.0,1.0,2.0
255583,-1.0,0.0,0.0,1.0,1.0
255584,1.0,0.0,0.0,1.0,0.0


In [117]:
print(data.shape)
data.drop_duplicates().shape

(255586, 5)


(255586, 5)

In [118]:
# data = data.drop_duplicates(subset=['val1', 'val2', 'val3'])
data.to_csv("../Data_Generation/Data_files/bfs_heisenberg_data.csv", index=False)

: 

In [94]:
duplicate_rows = data[data.duplicated(keep=False)]
duplicate_rows

,val1,val2,val3,num_steps,last_matrix
